# Numerical Methods - Week 3
## Kiran Shila - U54532811

In this notebook, I will demonstrate the deliverables for the week 3 assignment.

## Gaussian Elimination
The first part of this assignment was to write gaussian elimination code that utilizes partial pivoting

In [19]:
using BenchmarkTools
using LinearAlgebra

In [2]:
function gauss_elim(A::Matrix,b::Vector)
    # Create augmented matrix
    # make sure its a float otherwise we get float errors
    n = size(b)[1]
    A_Aug = float([A b])
    # For every column except the last one because pivots
    for k = 1:n-1
        # Find the largest pivot in this column, referenced to the pivot
        location = findmax(broadcast(abs,A_Aug[k:end,k]))[2] + k - 1
        maxVal = A_Aug[location,k]
        # Check to see if current pivot is the max,
        # if it isn't - swap
        if A_Aug[k,k] != maxVal
            for j in k:n+1
                A_Aug[location,j], A_Aug[k,j] = A_Aug[k,j], A_Aug[location,j]
            end
        end
        # Now perform gaussian elimination
        # For every row under the pivot
        for i = k+1:n
            # Normalize to the pivot and subtract from pivot row
            if A_Aug[i,k] != 0 # Ensures that we need to perform elimination
                scale = A_Aug[k,k] / A_Aug[i,k]
                for j in k:n+1 # Start at this column to the end including augmentation
                    A_Aug[i,j] = A_Aug[i,j] * scale
                    A_Aug[i,j] = A_Aug[i,j] - A_Aug[k,j]
                    if abs(A_Aug[i,j]) < eps()
                        A_Aug[i,j] = 0 # For stability against epsilon
                    end
                end
            end
        end
    end
    # Now back substitute to get to x
    x = zeros(Float64,n) # Create zeros for solution vector
    x[n] = A_Aug[end,end] / A_Aug[n,n] # Set the starting x
    # For every row, working backwards starting with the second from the bottom
    for i in n-1:-1:1
        x[i] = (A_Aug[i,end] - sum( [x[j] * A_Aug[i,j] for j in i+1:n] )) / A_Aug[i,i]
    end
    # Return solution vector x and U
    return x, A_Aug[:,1:end-1]
end

gauss_elim (generic function with 1 method)

Now to test against the examples from the assignment:
\begin{equation*}
\mathbf{A} =  \begin{vmatrix}
1 & 2 & 1\\
3 & 8 & 1\\
0 & 4 & 1
\end{vmatrix}
\end{equation*}

\begin{equation*}
\mathbf{b} =  \begin{vmatrix}
2 \\ 12 \\ 2
\end{vmatrix}
\end{equation*}

In [3]:
A = [1 2 1; 3 8 1; 0 4 1]
b = [2;12;2]
x,U = gauss_elim(A,b)
x

3-element Array{Float64,1}:
  2.0
  1.0
 -2.0

This matches what was expected from the assignment. Just to compare to the built-in solver:

In [4]:
A\b

3-element Array{Float64,1}:
  2.0
  1.0
 -2.0

And checking the upper triangular that we used to perform back-substitution

In [5]:
U

3×3 Array{Float64,2}:
 3.0  8.0   1.0
 0.0  4.0   1.0
 0.0  0.0  -5.0

Trying out the other example:
\begin{equation*}
\mathbf{A} =  \begin{vmatrix}
2 & 6 & 10\\
1 & 3 & 3\\
3 & 14 & 28
\end{vmatrix}
\end{equation*}

\begin{equation*}
\mathbf{b} =  \begin{vmatrix}
0 \\ 2 \\ -8
\end{vmatrix}
\end{equation*}

In [6]:
A = [2 6 10;1 3 3;3 14 28]
b = [0;2;-8]
x,U = gauss_elim(A,b)
x

3-element Array{Float64,1}:
  2.0
  1.0
 -1.0

In [7]:
U

3×3 Array{Float64,2}:
 3.0  14.0   28.0
 0.0  -5.0  -19.0
 0.0   0.0    6.0

In [8]:
A\b

3-element Array{Float64,1}:
  2.0000000000000013
  0.9999999999999992
 -0.9999999999999998

Interestingly enough, our solver was more stable than the built-in. Lets compare time, though

In [9]:
@benchmark gauss_elim(A,b)

BenchmarkTools.Trial: 
  memory estimate:  1.78 KiB
  allocs estimate:  39
  --------------
  minimum time:     1.960 μs (0.00% GC)
  median time:      2.043 μs (0.00% GC)
  mean time:        2.925 μs (26.68% GC)
  maximum time:     5.525 ms (99.92% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [10]:
@benchmark A\b

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  7
  --------------
  minimum time:     633.391 ns (0.00% GC)
  median time:      664.991 ns (0.00% GC)
  mean time:        776.964 ns (11.00% GC)
  maximum time:     290.033 μs (99.74% GC)
  --------------
  samples:          10000
  evals/sample:     169

Our code sucks. It is about 4 time slower than the built-in solver and uses 3 times more RAM.

The last part was to compare with computing the inverse. Given

Trying out the other example:
\begin{equation*}
\mathbf{A} =  \begin{vmatrix}
1 & 2 & 1\\
3 & 8 & 1\\
0 & 4 & 1
\end{vmatrix}
\end{equation*}

\begin{equation*}
\mathbf{b} =  \begin{vmatrix}
2 \\ 12 \\ 2
\end{vmatrix}
\end{equation*}

First lets make sure the sovler works.

In [15]:
A = [1 2 1;3 8 1;0 4 1]
b = [2;12;2]
x, U = gauss_elim(A,b)
x - (A\b) == zeros(length(b)) # Is our result correct?

true

Now compare performace with built-ins and inverse

In [16]:
@benchmark gauss_elim(A,b)

BenchmarkTools.Trial: 
  memory estimate:  1.78 KiB
  allocs estimate:  39
  --------------
  minimum time:     1.952 μs (0.00% GC)
  median time:      2.047 μs (0.00% GC)
  mean time:        2.778 μs (23.25% GC)
  maximum time:     5.095 ms (99.91% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [17]:
@benchmark A\b

BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  4
  --------------
  minimum time:     337.301 ns (0.00% GC)
  median time:      356.493 ns (0.00% GC)
  mean time:        443.913 ns (15.14% GC)
  maximum time:     345.616 μs (99.83% GC)
  --------------
  samples:          10000
  evals/sample:     219

In [18]:
@benchmark A^-1*b

BenchmarkTools.Trial: 
  memory estimate:  2.36 KiB
  allocs estimate:  8
  --------------
  minimum time:     873.315 ns (0.00% GC)
  median time:      922.778 ns (0.00% GC)
  mean time:        1.209 μs (20.48% GC)
  maximum time:     1.047 ms (99.87% GC)
  --------------
  samples:          10000
  evals/sample:     54

So the `backslash` operator was still the best, but our code had better memory efficiency than the inverse. The inverse was still faster.

## QR Decomposition
Just testing built-ins. Julia doesn't natively have magic() so I am just going to use randoms.

In [23]:
N = 3
A = rand(N,N)
Q,R = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.23512    0.884326   0.403343
 -0.634972   0.174432  -0.752585
 -0.735887  -0.433059   0.52051 
R factor:
3×3 Array{Float64,2}:
 -1.03476  -0.798556  -0.91775 
  0.0       0.627488   0.59903 
  0.0       0.0        0.307118

In [28]:
isapprox(Q*R,A) # Using approx for stability reasons

true

Hey look its a normal decomposition

## Eigenvalues
Just testing built-ins. Julia doesn't natively have magic() so I am just going to use randoms.

In [32]:
N = 3
A = rand(N,N)
eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
  1.5901676244663618 
 -0.2019847913538348 
  0.32939006548892424
eigenvectors:
3×3 Array{Float64,2}:
 0.318139   0.763281   0.357744
 0.71043   -0.467275  -0.921159
 0.627755  -0.446157   0.153251

Neat.